In [70]:
import math
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

gva = ["Vancouver","Surrey","Burnaby","Richmond","Coquitlam","Langley","Delta","North Vancouver","Maple Ridge","New Westminster","Port Coquitlam","North Vancouver","West Vancouver","Port Moody","Langley","White Rock","Pitt Meadows","Electoral Area A", "Bowen Island", "Anmore", "Lions Bay","Tsawwassen", "Belcarra"]
geolocator = Nominatim(user_agent="van", timeout=None)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [71]:
def geolocate(x):
    l = geolocator.geocode(x) # takes adress or city or lat long
    return l # return adress, negighborhood, city, postal, lat lon

In [72]:
def locate(x):
    #Variables
    h,s,c,p,lat,lon = x['House'],x['Street'],x['City'],x['PostalCode'],x['Latitude'],x['Longitude']
    
    #Truth Values
    hna = not x['House'] is 'nan' and not x['Street'] is 'nan'
    city = not x['City'] is 'nan'
    postal = not x['PostalCode'] is 'nan'
    latLonNull = not math.isnan(x['Latitude']) and not math.isnan(x['Longitude'])
    
    if hna and not latLonNull:
        if city:
            l = geolocate("{} {} {}".format(h,s,c))
            if not l is None:
                x['Latitude'],x['Longitude'] = l.latitude, l.longitude
        else:
            l = geolocate("{} {}".format(h,s))
            if not l is None:
                x['Latitude'],x['Longitude'] = l.latitude, l.longitude
    
    elif city and not latLonNull:
        l = geolocate("{}".format(c))
        if not l is None:
                x['Latitude'],x['Longitude'] = l.latitude, l.longitude
        
    if hna and city and not postal:
        l = geolocate("{} {} {}".format(h,s, c))
        if not l is None:
            x['PostalCode'] = l.raw['display_name'].split(', ')[5]
    
    return x

In [79]:
#Datasets for business licenses found at: https://data.vancouver.ca/datacatalogue/businessLicence.htm)
van_biz18 = pd.read_csv("2018business_licences.csv")
van_biz19 = pd.read_csv("business_licences.csv")
van_biz = van_biz18.append(van_biz19)
van_biz = van_biz.drop_duplicates(subset="LicenceNumber")

newbiz = van_biz.iloc[:,5:-2]
newbiz = newbiz[newbiz['Status'] != "Pending"]
newbiz = newbiz[newbiz['Province'] == 'BC']
newbiz = newbiz.drop(['Province','Country','NumberOfEmployees', "IssuedDate", "ExpiredDate", "Unit", "UnitType"], axis=1)
newbiz['City'] = newbiz['City'].apply(lambda x: "Vancouver" if str(x).lower()[0] == 'v' else x)
newbiz = newbiz[newbiz['City'].isin(gva)]

In [81]:
newbiz


,Status,BusinessType,BusinessSubType,House,Street,City,PostalCode,LocalArea,Latitude,Longitude
3,Issued,Apartment House,NaN,905,Chilco St,Vancouver,V6G 2R3,01-West End,NaN,NaN
5,Issued,Apartment House,NaN,117,W 13th Av,Burnaby,V5Y 1V8,09-Mount Pleasant,NaN,NaN
7,Inactive,Auto Detailing,NaN,NaN,NaN,White Rock,NaN,NaN,NaN,NaN
8,Issued,Auto Detailing,NaN,NaN,NaN,Burnaby,NaN,NaN,NaN,NaN
9,Issued,Auto Detailing,NaN,NaN,NaN,Coquitlam,NaN,NaN,NaN,NaN
12,Issued,Apartment House Strata,NaN,NaN,NaN,Vancouver,V6E 4N1,02-Central Business/Downtown,NaN,NaN
13,Issued,Apartment House Strata,NaN,NaN,NaN,Vancouver,V6E 3Z8,01-West End,NaN,NaN
14,Issued,Animal Services,Animal Control,NaN,NaN,Burnaby,NaN,NaN,NaN,NaN
15,Issued,Animal Services,Animal Walking,NaN,NaN,Richmond,NaN,NaN,NaN,NaN
16,Issued,Animal Services,Animal Sitting,NaN,NaN,Delta,NaN,NaN,NaN,NaN
